In [2]:
import json
import numpy as np
import pandas as pd
import os

In [8]:
def convert_str_to_float(string):
    num, denom = string.split('/')
    return float(num)/float(denom)

def pad_dict(vote_dict):
    for key in vote_dict:
        print key
        print vote_dict[key]
        #print np.isnan(vote_dict[key])
        if len(vote_dict[key]) != len(vote_dict['isAmendment']):
            vote_dict[key] = vote_dict[key] + [np.nan]*(len(vote_dict['isAmendment']) - len(vote_dict[key]))
    return vote_dict

In [11]:
congress_no = 1

content_list = []

for content in os.listdir("data/"+str(congress_no)+ "/votes"):
    if content != '.DS_Store':
        content_list.append(content)

def get_vote_nums(path):
    housevotes = []
    senvotes = []
    for folder in os.listdir(path):
        if folder[0] == 'h':
            housevotes.append(int(folder[1:]))
        elif folder[0] == 's':
            senvotes.append(int(folder[1:]))
    return housevotes[0], housevotes[len(housevotes)-1], senvotes[0], senvotes[len(senvotes)-1]
        
        
house_dict = {}
senate_dict = {}
for year in content_list:
    house_count = 0
    senate_count = 0
    path = "data/"+str(congress_no) + "/votes/" + year
    hstart, hend, sstart, send = get_vote_nums(path)
#     for folder in os.listdir(path):
#         if folder[0] == 'h':
#             house_count += 1
#         elif folder[0] == 's':
#             senate_count += 1
#     print year
#     print house_count
#     print senate_count
    house = get_vote_data(path+"/h", hstart, hend, house_dict)
    senate = get_vote_data(path+"/s", sstart, send, senate_dict)

house_dict = pad_dict(house_dict)
senate_dict = pad_dict(senate_dict)

df = pd.DataFrame(data=house_dict)
df.to_csv(str(congress_no)+"house.csv", index=False)

df = pd.DataFrame(data=senate_dict)
df.to_csv(str(congress_no)+"senate.csv", index=False)
#     hvotes = get_votes(path+"/h", house_count)
#     svotes = get_votes(path+"/s", senate_count)


data/1/votes/3/h87/data.json
unknown
S000349
['Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Not Voting', 'Nay', 'Yea', 'Nay', 'Nay', 'Nay', 'Yea', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Yea', 'Yea', 'Yea', 'Nay', 'Yea', 'Yea', 'Nay']
G000286
['Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Nay', 'Not Voting', 'Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Nay', 'Nay', 'Nay', 'Yea', 'Yea', 'Yea', 'Nay']
G000500
['Yea', 'Nay', 'Yea', 'Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Yea', 'Yea', 'Yea', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Nay', 'Nay', 'Nay', 'Not Voting', 'Yea', 'Nay', 'Yea']
S000222
['Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Nay', 'Yea', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'Not Voting', 'No

In [10]:
votes = ['Yea', 'Nay', 'Present', 'Not Voting']
res2 = []
def get_vote_data(path, start, end, vote_dict):
    for i in range(start, end+1):
        votepath = path + str(i) + "/data.json"
        f = open(votepath, 'r')
        vote = json.loads(f.read(), 'utf-8')
        if 'Aye' in vote['votes']:
            vote['votes']['Yea'] = vote['votes']['Aye']
        if 'No' in vote['votes']:
            vote['votes']['Nay'] = vote['votes']['No']
        if not any(vote_dict):
            if vote['category'] == 'passage' or vote['category'] == 'amendment' or vote['category'] == 'unknown':
                res2.append(i)
                vote_dict['date'] = [vote['date']]
                print votepath
                #vote_dict['billTitle'] = [vote['subject']]
                if vote['category'] == 'passage' or vote['category'] == 'unknown':
                    if 'bill' not in vote or vote['bill'] == None:
                        vote_dict['title'] = [vote['chamber'] + str(vote['number'])]
                    else:
                        vote_dict['title'] = [vote['bill']['type'] + str(vote['bill']['number'])]
                    vote_dict['isAmendment'] = [0]
                else:
                    vote_dict['title'] = [vote['amendment']['type'] + "amdt" + str(vote['amendment']['number']) + " to " + vote['bill']['type'] + str(vote['bill']['number'])]
                    vote_dict['isAmendment'] = [1]
                if 'result_text' not in vote:
                    vote_dict['result'] = [vote['result']]
                else:
                    vote_dict['result'] = [vote['result_text']]
                print vote['requires']
                if vote['requires'] == 'unknown':
                    vote_dict['requires'] = [np.nan]
                elif vote['requires'] == 'QUORUM':
                    vote_dict['requires'] = [convert_str_to_float("1/2")] # assuming quorum is simple 1/2 majority
                else:
                    vote_dict['requires'] = [convert_str_to_float(vote['requires'])]
                for vote_type in votes:
                    if vote_type in vote['votes']:
                        for person in vote['votes'][vote_type]:
                            vote_dict[person['id']] = [vote_type]
        else:
            if vote['category'] == 'passage' or vote['category'] == 'amendment' or vote['category'] == 'unknown': 
                res2.append(i)
                vote_dict['date'].append(vote['date'])
                #vote_dict['billTitle'].append(vote['subject'])
                if vote['category'] == 'passage' or vote['category'] == 'unknown':
                    if 'bill' not in vote or vote['bill'] == None:
                        vote_dict['title'].append(vote['chamber'] + str(vote['number']))
                    else:
                        #print votepath, vote['bill']
                        vote_dict['title'].append(vote['bill']['type'] + str(vote['bill']['number']))
                    vote_dict['isAmendment'].append(0)
                else:
                    vote_dict['title'].append(vote['amendment']['type'] + "amdt" + str(vote['amendment']['number']) + " to " + vote['bill']['type'] + str(vote['bill']['number']))
                    vote_dict['isAmendment'].append(1)
                if 'result_text' not in vote:
                    vote_dict['result'].append(vote['result'])
                else:
                    vote_dict['result'].append(vote['result_text'])
                
                if vote['requires'] == 'unknown':
                    vote_dict['requires'].append(np.nan)
                elif vote['requires'] == 'QUORUM':
                    vote_dict['requires'].append(convert_str_to_float("1/2")) # assuming quorum is simple 1/2 majority
                else:
                    vote_dict['requires'].append(convert_str_to_float(vote['requires']))
                for vote_type in votes:
                    if vote_type in vote['votes']:
                        for person in vote['votes'][vote_type]:
                            if person == 'VP':
                                print "The VP voted to break a tie in this vote"
                            elif person['id'] in vote_dict:
                                vote_dict[person['id']].append(vote_type)
                            else:
                                vote_dict[person['id']] = [np.nan]*(len(vote_dict['isAmendment'])-1) + [vote_type]
    return vote_dict

test = get_vote_data('data/109/votes/2005/h', 1, 671, {})
print len(test['isAmendment'])
print len(res2)

data/109/votes/2005/h1/data.json
QUORUM
391
391


In [29]:
print res2
print res

[11, 13, 16, 17, 18, 19, 21, 34, 35, 62, 72, 73, 74, 75, 77, 78, 88, 93, 101, 163, 187, 212, 213, 214, 230, 231, 244, 245, 246, 247, 248, 254]
[11, 13, 16, 17, 18, 19, 21, 34, 35, 62, 72, 73, 74, 75, 77, 78, 88, 93, 101, 163, 187, 212, 213, 214, 230, 231, 244, 245, 246, 247, 248, 254]


In [ ]:
def get_votes(path, count):
    res = []
    for i in range(1, count+1):
        votepath = path + str(i) + "/data.json"
        f = open(path, 'r')
        vote = json.loads(f.read())
        if vote['category'] == 'passage' or vote['category'] == 'amendment':
            res.append(i)
            #actually do here is call your dict function or whatever
    return res

In [19]:
res = []
#I might be interested in votes with categories like ['passage-suspension', 'treaty', 'conviction', 'veto-override']
for i in range(1, 261):
    path = '113/votes/2014/s' + str(i) + '/data.json'
    f = open(path, 'r')
    x = json.loads(f.read())
    if x['category'] == 'passage' or x['category'] == 'amendment':
        res.append(i)
        
print len(res)

32


In [23]:
print res
init_dict = True
votes = ['Yea', 'Nay', 'Present', 'Not Voting']
for index in res:
    path = '113/votes/2014/s' + str(index) + '/data.json'
    vote = json.loads(open(path, 'r').read())
    if init_dict:
        vote_dict = {}
        vote_dict['date'] = [vote['date']]
#         vote_dict['billTitle'] = [vote['subject']]
        if vote['category'] == 'passage':
            vote_dict['title'] = [vote['bill']['type'] + str(vote['bill']['number'])]
            vote_dict['isAmendment'] = [0]
        else:
            vote_dict['title'] = [vote['amendment']['type'] + "amdt" + str(vote['amendment']['number']) + " to " + vote['bill']['type'] + str(vote['bill']['number'])]
            vote_dict['isAmendment'] = [1]
        vote_dict['result'] = [vote['result_text']]
        vote_dict['requires'] = [convert_str_to_float(vote['requires'])]
        for vote_type in votes:
            for person in vote['votes'][vote_type]:
                vote_dict[person['display_name']] = [vote_type]
        init_dict = False
    else:
        vote_dict['date'].append(vote['date'])
#         vote_dict['billTitle'].append(vote['subject'])
        if vote['category'] == 'passage':
            vote_dict['title'].append(vote['bill']['type'] + str(vote['bill']['number']))
            vote_dict['isAmendment'].append(0)
        else:
            vote_dict['title'].append(vote['amendment']['type'] + "amdt" + str(vote['amendment']['number']) + " to " + vote['bill']['type'] + str(vote['bill']['number']))
            vote_dict['isAmendment'].append(1)
        vote_dict['result'].append(vote['result_text'])
        vote_dict['requires'].append(convert_str_to_float(vote['requires']))
        for vote_type in votes:
            for person in vote['votes'][vote_type]:
                if person['display_name'] in vote_dict:
                     vote_dict[person['display_name']].append(vote_type)
                else:
                    vote_dict[person['display_name']] = [np.nan]*(len(vote_dict['isAmendment'])-1) + [vote_type]

#pad with nones to make all lists the same size

for key in vote_dict:
    if len(vote_dict[key]) != len(vote_dict['isAmendment']):
        vote_dict[key] = vote_dict[key] + [np.nan]*(len(vote_dict['isAmendment']) - len(vote_dict[key]))

import pandas as pd
df = pd.DataFrame(data=vote_dict)
df.to_csv('rollCallVotes_iter1.csv', index=False)

[11, 13, 16, 17, 18, 19, 21, 34, 35, 62, 72, 73, 74, 75, 77, 78, 88, 93, 101, 163, 187, 212, 213, 214, 230, 231, 244, 245, 246, 247, 248, 254]


In [24]:
print len(vote_dict['isAmendment'])
print len(test['isAmendment'])

32
33
